# Desmascarando Robôs
### CRISP-DM Cycle 1
---

Imagine um mercado online, um palco digital onde diversos leilões se desenrolam a cada segundo. Neste ambiente, participantes do mundo inteiram lançam seus lances em busca de objetos desejados, desde joias até equipamentos tecnológicos. No entanto, nem todos os jogadores neste campo são humanos; alguns são robôs programados para manipular os resultados dos leilões.

Seu desafio é se aprofundar nesses dados, explorar as camadas de atividade nos leilões e conseguir construir um modelo que saiba muito bem diferenciar humanos de robôs.

> Disclaimer: This is a fictional bussiness cas

## 0. PREPARATION

### 0.1 Settings

In [1]:
# Settings imports
import os
import sys
import pandas as pd
from dotenv import load_dotenv

# Load .env file
env_path = "../.env"
load_dotenv(dotenv_path=env_path)

# Seed
seed = int(os.getenv("SEED"))

# Add path
path = os.getenv("HOMEPATH")

# Add path to sys.path
sys.path.append(path)

In [2]:
from helper.classes.FeatureEngineering import FeatureEngineering
import helper.functions.descriptive_statistics as ds

/Users/guttofranca/Repos/desmascarando-robos/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 0.2 Data

**Train e Test**

- **id_participante**: Identificador único do participante
- **conta_pagamento**: Conta de pagamento associada ao participante (com o valor ocultado) # Não será utilizada
- **endereco**: Endereço postal do participante # Não será utilizada
- **resultado**: A variável alvo que identifica se o participante é um robô ou um humano. (Robô = 1 e Humano = 0). (*target*)

- **Robôs Confirmados**: Participantes com provas claras de atividades fraudulentas, resultando em banimento da plataforma. São rotulados como robôs no conjunto de dados (resultado = 1).

- **Robôs Suspeitos**: Participantes com atividades atípicas ou estatísticas que superam a média, mas sem provas definitivas de fraude. A classificação deles como robôs é incerta.

**Lances**

- **id_lance**: Identificador único do lance
- **id_participante**: Identificador único do participante
- **leilao**: Identificador único do leilão 
- **mercadoria**: A categoria da mercadoria leiloada
- **dispositivo**: O dispositivo utilizado pelo visitante
- **tempo**: O tempo que o lance foi feito
- **pais**: O país que o IP pertence
- **ip**: O IP do participante
- **url**: A URL de onde o participante foi referido

## 1. DATA UNDERSTANDING

### 1.1 Loading Data

In [3]:
train = pd.read_feather(path + "/data/raw/train.feather")
test = pd.read_feather(path + "/data/raw/test.feather")
lances = pd.read_feather(path + "/data/raw/lances.feather")

In [4]:
train.sample().T

,938
id_participante,a116e727a3dc0b69f5e7ae14d0ab40d44breq
conta_pagamento,44a028c0c58e104e4600d44b0a0f5d92m1q1v
endereco,4d90f2e709f1fc0810e5aef472dd3935th27z
resultado,0


In [5]:
lances.sample().T

,994953
id_lance,3346148
id_participante,9328bc6389d718c41e33c4cefe7b9c15k1tau
leilao,ywmkf
mercadoria,bens domésticos
dispositivo,phone185
tempo,9636599263157894
pais,my
ip,91.162.27.152
url,6ag9vs3h4g4j253


In [6]:
df_train = pd.merge(train, lances, on="id_participante", how="inner")
df_test = pd.merge(test, lances, on="id_participante", how="inner")

### 1.2 Verify NA

In [7]:
df_train.isna().sum()

id_participante       0
conta_pagamento       0
endereco              0
resultado             0
id_lance              0
leilao                0
mercadoria            0
dispositivo           0
tempo                 0
pais               2236
ip                    0
url                   0
dtype: int64

Inicialmente, irei remover os valores nulos.

In [8]:
for data in [df_train, df_test]:
    data.dropna(inplace=True)

In [9]:
df_train.isna().sum()

id_participante    0
conta_pagamento    0
endereco           0
resultado          0
id_lance           0
leilao             0
mercadoria         0
dispositivo        0
tempo              0
pais               0
ip                 0
url                0
dtype: int64

In [10]:
df_test.isna().sum()

id_participante    0
conta_pagamento    0
endereco           0
id_lance           0
leilao             0
mercadoria         0
dispositivo        0
tempo              0
pais               0
ip                 0
url                0
dtype: int64

### 1.3 Data Shape

In [11]:
print(
    f"Train:\nO dataset possui {df_train.shape[0]} linhas e {df_train.shape[1]} colunas\n"
)
print(f"Test:\nO dataset possui {df_test.shape[0]} linhas e {df_test.shape[1]} colunas")

Train:
O dataset possui 2532089 linhas e 12 colunas

Test:
O dataset possui 536434 linhas e 11 colunas


### 1.4 Unique Values

Utilizarei apenas os dados de treino aqui, se existem vários valores únicos em treino, existem em teste.

In [12]:
# Código de escape ANSI para negrito
negrito = "\033[1m"
reset = "\033[0m"

for col in df_test.columns:
    print(
        f"Valores únicos em Train[{col}]: {negrito}{df_train[col].nunique()}{reset} valores"
    )
    print(
        f"Valores únicos em Test[{col}]: {negrito}{df_test[col].nunique()}{reset} valores"
    )

Valores únicos em Train[id_participante]: 1393 valores
Valores únicos em Test[id_participante]: 590 valores
Valores únicos em Train[conta_pagamento]: 1393 valores
Valores únicos em Test[conta_pagamento]: 590 valores
Valores únicos em Train[endereco]: 1393 valores
Valores únicos em Test[endereco]: 590 valores
Valores únicos em Train[id_lance]: 2532089 valores
Valores únicos em Test[id_lance]: 536434 valores
Valores únicos em Train[leilao]: 12199 valores
Valores únicos em Test[leilao]: 7707 valores
Valores únicos em Train[mercadoria]: 9 valores
Valores únicos em Test[mercadoria]: 8 valores
Valores únicos em Train[dispositivo]: 5391 valores
Valores únicos em Test[dispositivo]: 3477 valores
Valores únicos em Train[tempo]: 720004 valores
Valores únicos em Test[tempo]: 367521 valores
Valores únicos em Train[pais]: 198 valores
Valores únicos em Test[pais]: 178 valores
Valores únicos em Train[ip]: 855829 valores
Valores únicos em Test[ip]: 226254 valores
Valores únicos em Train[url]: 535408 va

### 1.5 Data Types

In [13]:
df_train.dtypes

id_participante    object
conta_pagamento    object
endereco           object
resultado           int64
id_lance            int64
leilao             object
mercadoria         object
dispositivo        object
tempo               int64
pais               object
ip                 object
url                object
dtype: object

In [14]:
df_test.dtypes

id_participante    object
conta_pagamento    object
endereco           object
id_lance            int64
leilao             object
mercadoria         object
dispositivo        object
tempo               int64
pais               object
ip                 object
url                object
dtype: object

In [15]:
# Converter tempo para datetime
df_train["tempo"] = pd.to_datetime(df_train["tempo"])
df_test["tempo"] = pd.to_datetime(df_test["tempo"])

### 1.6 Tempo

A coluna `tempo` precisa de um pouco de atenção, principalmente na conversão para datetime

In [16]:
# Verificar se todas estão no mesmo ano
print(f"Train Anos: {df_train["tempo"].dt.year.unique()}")
print(f"Test Anos: {df_test["tempo"].dt.year.unique()}")

# Verificar se todas estão no mesmo mês
print(f"Train Meses: {df_train["tempo"].dt.month.unique()}")
print(f"Test Meses: {df_test["tempo"].dt.month.unique()}")

# Verificar se todas estão no mesmo dia
print(f"Train Dias: {df_train["tempo"].dt.day.unique()}")
print(f"Test Dias: {df_test["tempo"].dt.day.unique()}")

Train Anos: [1970]
Test Anos: [1970]
Train Meses: [4]
Test Meses: [4]
Train Dias: [22 23 24]
Test Dias: [22 23 24]


O ano e o mês são os mesmos, então não precisamos deles. Usaremos isso no futuro.

### 1.7 Pais

A coluna `pais` está utilizando a ISO 3166-1 alpha-2 para identificar os países com códigos de duas letras, irei atualizar todos para esse padrão.

In [17]:
df_train.pais.unique()

array(['Índia', 'tr', 'ro', 'my', 'br', 'ca', 'Indonésia',
       'Estados Unidos', 'Tailândia', 'sg', 'cl', 'Rússia', 'bo', 'nl',
       'at', 'se', 'si', 'bn', 'sz', 'de', 'lt', 'mx', 'ec', 'cz', 'fi',
       'hr', 'Nigéria', 'ch', 'iq', 'África do Sul', 'gt', 'gh', 'gr',
       'Paraguai', 'pl', 'ua', 'ke', 'bg', 'ie', 'rs', 'li', 'hu', 'ph',
       'es', 'dk', 'nz', 'it', 'be', 'uk', 'cn', 'ar', 'mk', 'Austrália',
       'sk', 'fr', 'no', 'vn', 'ni', 'pe', 'pk', 'dz', 'bd', 'kw', 'bh',
       'sa', 'il', 'jo', 'tt', 'om', 'lk', 'pa', 'sv', 'lb', 'co', 'qa',
       'ae', 'az', 'pr', 'ma', 'ge', 'ml', 'gq', 'zw', 'eg', 'et', 'dj',
       'tg', 'eu', 'np', 'tz', 'sl', 'cm', 'zm', 'bj', 'lr', 'ga', 'bw',
       'ug', 'mw', 'hk', 'cg', 'ne', 'mu', 'cd', 'mz', 'sn', 'na', 'ba',
       'rw', 'lu', 'ao', 'tn', 'af', 'sr', 'jm', 'ir', 'Sudão', 'tj',
       'ly', 've', 'gm', 'pt', 'mg', 'lv', 'tw', 'am', 'me', 'md', 'bf',
       'mr', 'ee', 'cy', 'al', 'ls', 'so', 'ad', 'by', 'td', 'mc', 'is

In [18]:
df_test.pais.unique()

array(['my', 'rs', 'ke', 'África do Sul', 'Índia', 'lk', 'Tailândia',
       'cn', 'ae', 'sg', 'Indonésia', 'sa', 'pk', 'ma', 'Nigéria', 'tr',
       'de', 'lb', 'ye', 'bn', 'Estados Unidos', 'qa', 'bd', 'vn', 'ir',
       'eg', 'om', 'az', 'ro', 'np', 'uk', 'Sudão', 'Rússia', 'gt', 'kw',
       'et', 'ca', 'ph', 'no', 'tz', 'ly', 'eu', 'nz', 'hk', 'bh', 'es',
       'kz', 'Austrália', 'kg', 'tn', 'jo', 'mv', 'gh', 'pl', 'hr', 'ml',
       'jp', 'tw', 'ar', 'na', 'ao', 'sz', 'dz', 'mw', 'lt', 'se', 'ls',
       'be', 'br', 'dj', 'at', 'md', 'uz', 've', 'kh', 'pa', 'Paraguai',
       'cz', 'it', 'so', 'fr', 'dk', 'ua', 'ge', 'af', 'nl', 'mn', 'bt',
       'co', 'zw', 'ci', 'gq', 'cv', 'mu', 'gr', 'mz', 'me', 'ug', 'si',
       'sc', 'ch', 'ie', 'sl', 'hu', 'zm', 'lr', 'rw', 'bj', 'ec', 'ba',
       'cy', 'al', 'ne', 'pt', 'mc', 'cm', 'mr', 'mk', 'mg', 'td', 'iq',
       'bf', 'pe', 'bi', 'gm', 'by', 'bg', 'sk', 'bw', 'bo', 'am', 'ga',
       'sn', 'ee', 'tg', 'lu', 'mx', 'il', 'mm', 'tt

### 1.8 Dispositivo


In [19]:
df_train.dispositivo.unique()

array(['phone61', 'phone542', 'phone59', ..., 'phone7042', 'phone6921',
       'phone4883'], dtype=object)

In [20]:
df_train.dispositivo.nunique()

5391

### 1.9 Mercadoria

In [21]:
df_train.mercadoria.unique()

array(['joias', 'móveis', 'bens domésticos', 'artigos esportivos',
       'equipamentos de escritório', 'livros e música', 'computadores',
       'vestuário', 'peças de automóveis'], dtype=object)

### 1.10 URL

In [22]:
df_train.url.nunique()

535408

### 1.11 Resultado

In [23]:
ds.categorical_metrics(df_train, "resultado")

,absolute,percent %
resultado,,
0,2198890,86.840944
1,333199,13.159056


## 2. FEATURE ENGINEERING

In [24]:
fe = FeatureEngineering()

df_test = fe.perform_transformations(df_test)
X_train, X_test, X_val, y_train, y_test, y_val = fe.transform(df_train)

for data in [X_train, X_test, X_val]:
    data.drop(columns=["id_participante", "id_lance", "tempo", "ip"], inplace=True)

# Salvando os dados
X_train.to_feather(path + "/data/processed/X_train.feather")
X_test.to_feather(path + "/data/processed/X_test.feather")
X_val.to_feather(path + "/data/processed/X_val.feather")
y_train.to_pickle(path + "/data/processed/y_train.pkl")
y_test.to_pickle(path + "/data/processed/y_test.pkl")
y_val.to_pickle(path + "/data/processed/y_val.pkl")
df_test.to_feather(path + "/data/processed/final_test.feather")

In [25]:
df_test.duplicated().sum()

0

In [26]:
df_test.head()

,id_participante,conta_pagamento,endereco,id_lance,leilao,mercadoria,dispositivo,tempo,pais,ip,...,total_mercadorias,mercadoria_mais_frequente,dispositivo_mais_usado,total_dispositivos,total_paises,pais_mais_frequente,total_ips,ip_mais_frequente,total_urls,url_mais_frequente
0,a71f84e476d03bd0b25d45e44eeed9a8efehw,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv,2394993,8gshg,joias,phone469,1970-04-22 11:33:30.789473684,my,233.253.145.73,...,1,joias,phone49,117,7,my,189,136.111.77.221,61,8ptbt1q05i4h1lu
1,a71f84e476d03bd0b25d45e44eeed9a8efehw,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv,2401931,8gshg,joias,phone46,1970-04-22 11:33:46.315789473,my,155.48.112.213,...,1,joias,phone49,117,7,my,189,136.111.77.221,61,8ptbt1q05i4h1lu
2,a71f84e476d03bd0b25d45e44eeed9a8efehw,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv,2406130,ca59r,joias,phone46,1970-04-22 11:33:56.000000000,my,28.105.143.62,...,1,joias,phone49,117,7,my,189,136.111.77.221,61,8ptbt1q05i4h1lu
3,a71f84e476d03bd0b25d45e44eeed9a8efehw,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv,2452472,8gshg,joias,phone13,1970-04-22 11:35:52.421052631,my,56.49.2.96,...,1,joias,phone49,117,7,my,189,136.111.77.221,61,8ptbt1q05i4h1lu
4,a71f84e476d03bd0b25d45e44eeed9a8efehw,a3d2de7675556553a5f08e4c88d2c228h98zm,a3d2de7675556553a5f08e4c88d2c228f0bvv,2456348,8gshg,joias,phone524,1970-04-22 11:36:03.105263157,my,95.116.232.134,...,1,joias,phone49,117,7,my,189,136.111.77.221,61,8ptbt1q05i4h1lu


**Novas Features**

- **dia**: Dia da operação no leilão
- **hora**: Hora da operação no leilão
- **minuto**: Minuto da operação no leilão
- **segundo**: Segundo da operação no leilão
- **primeiro_octeto_ip**: O primeiro octeto do IP do participante
- **segundo_octeto_ip**: O segundo octeto do IP do participante
- **contagem_participante**: O número de lances feitos pelo participante
- **contagem_leilao**: O número de lances feitos no leilão
- **contagem_conta_pagamento**: O número de lances feitos pela conta de pagamento
- **frequencia_dispositivo**: A frequência do dispositivo utilizado pelo participante
- **horario_principal**: Se o horário da operação é o horário principal (entre 9h e 17h)
- **ip_classe**: A classe do IP do participante